In [1]:
import json
import random
import numpy as np
from tqdm import tqdm
from scipy import stats
from scipy.spatial.distance import cosine
from sentence_transformers import SentenceTransformer

In [2]:
## context sentences

with open('data/zt_ori_84.txt', 'r') as f:
    lines = f.readlines()
    names = [line.strip().split('\t')[0] for line in lines]
    zt_scores = [line.strip().split('\t')[-1] for line in lines]

with open('data/thermo_sens.json','r') as f1:
    string = f1.read()
    thermo_sents = json.loads(string)
    
with open('data/mat_sens.json','r', encoding='utf-8') as f2:
    string = f2.read()
    dictionary = json.loads(string)   # a dictionary of dictionaries
    
dict_84 = {}
for d in dictionary.values():
    if 'if_84' in d.keys():
        if 'sens' in d.keys():
            dict_84[d['ori']] = d['sens']
        else:
            dict_84[d['ori']] = d['ori']
# mat_without_sents = [key for key,value in dict_84.items() if key == value]
# mat_over_100 = [key for key,value in dict_84.items() if len(value) > 100]
# print(len(mat_without_sents), len(mat_over_100))
for key,value in dict_84.items():
    if len(value) > 100:
        dict_84[key] = random.sample(value, 100)
        # random.sample(population, k) chooses k unique random elements from a population sequence or set, returns a new list of sampled elements

# print(list(dict_84.keys()) == names)
# print(dict_84)

In [3]:
## embeddings and spearman correlation

def context_average(model):

    center_embedding = np.mean(model.encode(thermo_sents), axis=0)
    
    cos_sims = []
    for name in tqdm(names):
        if type(dict_84[name]) == list:
            encoded = model.encode(dict_84[name])
            embedding = np.mean(encoded, axis=0)
        else:
            embedding = model.encode(dict_84[name])
        cos_sim = 1-cosine(center_embedding, embedding)
        cos_sims.append(cos_sim)

    corr, pvalue = stats.spearmanr(cos_sims, zt_scores)
    return corr

In [ ]:
path = 'outputs/SentMatBERT_MNR'
SentMatBERT_MNR = SentenceTransformer(path)
print(context_average(SentMatBERT_MNR))   
# best result: 0.5923 with CPU, 0.5919 with GPU (first run in Vega)